In [14]:
# 데이터 전처리(정규변환,로그변환)을 해주는 함수 정의

from sklearn.preprocessing import StandardScaler,MinMaxScaler,PolynomialFeatures
import numpy as np
import pandas as pd

boston_df = pd.read_csv('./datasets/Boston.csv',index_col=0)

x = boston_df.drop(['medv'],axis=1)
y = boston_df['medv']


# method는 표준 정규 분포 변환(Standard),최대/최소 정규화(MinMax), 로그변환(log)결정
# p_degree 는 다항식 특성을 추가할 때 적용 , p_degree는 2이상 부여하지 않음
def get_scaled_data(method = 'None',p_degree=None,input_data=None):
    if method =='Standard':
        scaled_data = StandardScaler().fit_transform(input_data)
    elif method =='MinMax':
        scaled_data = MinMaxScaler().fit_transform(input_data)
    elif method =='Log':
        scaled_data =np.log1p(input_data)
    else :
        scaled_data = input_data
        
    
    if p_degree != None:
        scaled_data = PolynomialFeatures(degree=p_degree,include_bias=False).fit_transform(scaled_data)
        
    return scaled_data

In [18]:
from sklearn.linear_model import ElasticNet,Ridge,Lasso
from sklearn.model_selection import cross_val_score

def get_linear_reg_eval(model_name,params =None,x_data_n=None,y_target_n=None,verbose=True):
    coeff_df = pd.DataFrame()
    if verbose : print('######',model_name,'######')
    for param in params:
        if model_name =='Ridge' :
            model = Ridge(alpha=param)
        elif model_name =='Lasso' :
            model = Lasso(alpha=param)
        elif model_name =='ElasticNet' :
            model = ElasticNet(alpha=param,l1_ratio=0.7) 
        neg_mse_scores = cross_val_score(model,x_data_n,y_target_n,scoring='neg_mean_squared_error',cv=5)
        avg_rmse = np.mean(np.sqrt(-1*neg_mse_scores))
        print(f'alpha {param} 일 때 5폴드 세트의 평균 RMSE: {avg_rmse}')

In [20]:
# Ridge의 alpha 값을 다르게 적용하고 다양한 데이터 변환방법에 따른 rmse 추출
alpha = [0.1,1,10,100]

# 변환 방법
scale_methods = [(None,None),('Standard',None),('Standard',2),
               ('MinMax',None),('MinMax',2),('Log',None)]

for scale_method in scale_methods:
    x_data_scaled = get_scaled_data(method=scale_method[0],p_degree=scale_method[1],input_data=x)
    print(f'## 변환유형,{scale_method[0]},polynomial_degree,{scale_method[1]}')
    
    get_linear_reg_eval('Ridge',params=alpha,x_data_n=x_data_scaled,
                       y_target_n=y,verbose=False)

## 변환유형,None,polynomial_degree,None
alpha 0.1 일 때 5폴드 세트의 평균 RMSE: 5.788486627032412
alpha 1 일 때 5폴드 세트의 평균 RMSE: 5.652570965613549
alpha 10 일 때 5폴드 세트의 평균 RMSE: 5.518166280868973
alpha 100 일 때 5폴드 세트의 평균 RMSE: 5.329589628472148
## 변환유형,Standard,polynomial_degree,None
alpha 0.1 일 때 5폴드 세트의 평균 RMSE: 5.825992799389445
alpha 1 일 때 5폴드 세트의 평균 RMSE: 5.802889517257742
alpha 10 일 때 5폴드 세트의 평균 RMSE: 5.6368312225590715
alpha 100 일 때 5폴드 세트의 평균 RMSE: 5.421374711794966
## 변환유형,Standard,polynomial_degree,2
alpha 0.1 일 때 5폴드 세트의 평균 RMSE: 8.827235873001843
alpha 1 일 때 5폴드 세트의 평균 RMSE: 6.871287525962543
alpha 10 일 때 5폴드 세트의 평균 RMSE: 5.484870425533376
alpha 100 일 때 5폴드 세트의 평균 RMSE: 4.6344377782619235
## 변환유형,MinMax,polynomial_degree,None
alpha 0.1 일 때 5폴드 세트의 평균 RMSE: 5.763570225288312
alpha 1 일 때 5폴드 세트의 평균 RMSE: 5.465045081564942
alpha 10 일 때 5폴드 세트의 평균 RMSE: 5.754163637679423
alpha 100 일 때 5폴드 세트의 평균 RMSE: 7.63491986362354
## 변환유형,MinMax,polynomial_degree,2
alpha 0.1 일 때 5폴드 세트의 평균 RMSE: 5.29756000

array([[-0.41978194,  0.28482986, -1.2879095 , ..., -1.45900038,
         0.44105193, -1.0755623 ],
       [-0.41733926, -0.48772236, -0.59338101, ..., -0.30309415,
         0.44105193, -0.49243937],
       [-0.41734159, -0.48772236, -0.59338101, ..., -0.30309415,
         0.39642699, -1.2087274 ],
       ...,
       [-0.41344658, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.98304761],
       [-0.40776407, -0.48772236,  0.11573841, ...,  1.17646583,
         0.4032249 , -0.86530163],
       [-0.41500016, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.66905833]])